# Hardware demonstration: ECG anomaly detection

Author: Felix Bauer

This notebook demonstrates how recurrent SNNs can be used for anomaly detection in an electrocardiogram (ECG) signal. The main part of the network will run on a DYNAP-SE neuromorphic processor.

# Task

Our goal is to detect five different classes of anomalies in a two-lead ECG signal from the MIT-BIH Arrithmia Database [ref]. An SNN that fulfills this task can, for instance, be used in a wearable ECG monitoring device to trigger an alarm in presence of pathological patterns. Below we see examples for a normal ECG signal and for each anomaly type.

In [1]:
%matplotlib widget

from scripts.plot_example_beats import plot_examples, labels
plot_examples()

WARNING    /home/felix/miniconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
 [py.warnings]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Hardware

We will first run a software simulation of our SNN and then run the network directly on a DYNAP-SE neuromorphic processor. The device we are demonstrating here is a prototype that imposes a few restrictions on the network, which are described below and which will also be considered in the software simulations.

## Core-wise parameters
Neuron and synapse parameters, such as time constants, firing thresholds and weights are set per core. The present processor consists of 16 cores of 256 neurons each. 

## Discrete weights
Synaptic weights are the same for each postsynaptic neuron on a core, resulting in ternary weights: positive (excitatory), negative (inhibitory), and zero (not connected). However, between each pair of neurons, multiple connections are possible, which effectively allows for integer weights.

## Connectivity
The number of presynaptic connections to each neuron is generally limited to 64.



# Input data

To load the ECG data we will use a data loader class that is specifically written for this purpose. You find its source code in the folder of this tutorial. 
The ECG data itself is distributed on [PhysioNet](http://physionet.org/physiobank/database/mitdb/). We extracted the signal and its annotations to a .npy-file and a .csv file that can be downloaded from [here](https://www.dropbox.com/s/ocqo8oog0xv4qrm/ecg_data.zip?dl=1). If you want to run this notebook you need to extract the files and save them in the folder called `ecg_data`.

In [2]:
from scripts.dataloader import ECGDataLoader

# - Object to load ECG data
data_loader = ECGDataLoader()

ECG signal and annotaitions have been loaded from /home/felix/gitlab/amld-2020-aictx/Notebooks/ECG_demo/ecg_data


# Signal-to-spike encoding

The analog ECG signal is converted to trains of events through a sigma-delta encoding scheme. For every ECG lead there are two output channels, emitting spikes when the input signal increases ("up"-channel) or decreases ("down"-channel) by a specified amount.

The four resulting spike trains serve as input to the acutal network.

In [3]:
from rockpool.layers import FFUpDown

# - ECG signal parameters
DT_ECG = 0.002778
NUM_ECG_LEADS = 2
NUM_ANOM_CLASSES = 4

# - Spike encoding
spike_enc = FFUpDown(
    weights=NUM_ECG_LEADS,
    dt=DT_ECG,    
    thr_up=0.1,
    thr_down=0.1,
    multiplex_spikes=True,
    name="spike_encoder"
)

# Network architecture


To detect the anomalies we will use a partitioned reservoir network, consisting of 768 neurons. Each neuron is either excitatory or inhibitory. This is not strictly necessary but this way the different reservoir partitions can be placed onto different cores of the processor, which makes it easier to control the neuron dynamics.

The weight matrices for the hardware will have integer values, corresponding to the number of connections between each pair of neurons. For the software simulation we will use the same matrix but scaled, such that the weights have the right strength. Different partitions of the reservoir will get different weights. For the hardware reservoir the weights will be scaled later on by setting a hardware parameter.

In [4]:
import numpy as np

from rockpool.layers import RecIAFSpkInNest

# Load weights
weights_res_in = np.load("network/weights_res_in.npy")
weights_rec = np.load("network/weights_rec.npy")

# Scale weights for software simulation
start_rec = 128
start_inh = 128 + 512

baseweight_inp = 5e-4
baseweight_exp_rec = 8e-5
baseweight_rec = 8e-5  # 1.75e-4
baseweight_rec_inh = 8e-5
baseweight_inh = 1e-4

weights_res_in_scaled = weights_res_in.copy() * baseweight_inp

weights_rec_scaled = weights_rec.copy()
weights_rec_scaled[:start_rec, start_rec: start_inh] *= baseweight_exp_rec
weights_rec_scaled[start_rec: start_inh, start_rec: start_inh] *= baseweight_rec
weights_rec_scaled[start_rec: start_inh, start_inh:] *= baseweight_rec_inh
weights_rec_scaled[start_inh:, start_rec: start_inh] *= baseweight_inh

# - Load reservoir parameters from file (generated with gen_params.py)
kwargs_reservoir = dict(np.load("network/kwargs_reservoir.npz"))

# - Instantiate reservoir layer object
reservoir = RecIAFSpkInNest(
    weights_in=weights_res_in_scaled,
    weights_rec=weights_rec_scaled,
    name="reservoir",
    **kwargs_reservoir
)

WARNING    /home/felix/miniconda3/lib/python3.7/site-packages/nest/lib/hl_api_helper.py:127: UserWarning:
ResetNetwork is deprecated and will be removed in NEST 3.0.
 [py.warnings]


## Readout layer

The readout layer low-pass filters the reservoir spike trains to obtain an analog signal. It is then trained by ridge regression to perform a linear separation between the ECG anomaly types. There is one readout unit for each anomaly and the corresponding target is 1 whenever the anomaly is present and 0 otherwise.

In [5]:
from rockpool.layers import FFExpSyn

readout = FFExpSyn(
    weights=np.zeros((reservoir.size, NUM_ANOM_CLASSES)),
    bias=0,
    dt=DT_ECG,
    tau_syn=0.175,
    name="readout"
)

## Network

In [6]:
from rockpool import Network

# - Network that holds the layers
sw_net = Network(spike_enc, reservoir, readout, dt=DT_ECG)

# Software Simulation

## Training
In the following we will train the (software) readout layer. For this we generate batches of ECG data and evolve the network with it as input. After each batch the readout weights are updated. Although the linear regression algorithm is traditionally not for classification tasks, we use it here because it is very fast and efficient.

_(We have trained the readout beforehand and will simply load the weights)_

In [ ]:
# import time

# # - Generator that yields batches of ECG data
# batchsize_training = 1000
# num_beats = 15000
# regularize = 0.1
# batch_gen = data_loader.get_batch_generator(num_beats=num_beats, batchsize=batchsize_training)

# t_start = time.time()
# for batch in batch_gen:
#     output = sw_net.evolve(batch.input)
#     readout.train_rr(
#         batch.target,
#         output["reservoir"],
#         is_first=batch.is_first,
#         is_last=batch.is_last,
#         regularize=regularize,
#     )
    
# sw_net.reset_all()
# print(f"Trained network in {time.time() - t_start:.2f} seconds.")

# np.save("network/readout_weights.npy", readout.weights)
# np.save("network/readout_bias.npy", readout.bias)

In [ ]:
# # - Test on training data
# target = batch.target.start_at_zero()
# res_data = output["reservoir"]

# test_on_training = readout.evolve(res_data.start_at_zero())
# readout.reset_all()

In [ ]:
# %matplotlib widget
# from rockpool import TSContinuous
# test_on_training.clip(channels=3).plot()
# target.clip(channels=3).plot()
# any_target = TSContinuous(target.times, np.any(target.samples, axis=1))
# any_target.plot(color="gray", alpha=0.5)

In [7]:
readout.weights = np.load("network/readout_weights.npy")
readout.bias = np.load("network/readout_bias.npy")

## Inference

We can now test our network to see how it performs with data it has not been trained on.
The plots below show the output of each readout unit (blue). The targets are plotted in orange. 

Because the readout units are only trained to distinguish between normal and one specific anomaly, there many cross-detections. We therefore also plot a gray curve that indicates the presence of _any_ anomaly. For many applications it is sufficient to know that there is an anomaly. If one needs to classify which type it is, one could, for instnace, train an all-vs-all classifier.

In [8]:
# - Generator that yields batches of ECG data
num_beats = 100
ecg_data = data_loader.get_single_batch(num_beats=num_beats)

net_data = sw_net.evolve(ecg_data.input)
output = net_data["readout"]
sw_net.reset_all()

Network: Evolving layer `spike_encoder` with external input as input
Network: Evolving layer `reservoir` with spike_encoder's output as input
Network: Evolving layer `readout` with reservoir's output as input


In [9]:
%matplotlib widget
from matplotlib import pyplot as plt
from rockpool import TSContinuous

target = ecg_data.target
any_target = TSContinuous(target.times, np.any(target.samples, axis=1))

fig, axes = plt.subplots(2, 2, figsize=(8, 6))
plt.subplots_adjust(
    top=0.95, bottom=0.05, left=0.05, right=0.95, hspace=0.2, wspace=0.2
)

for i_anom, (ax, lbl) in enumerate(zip(axes.flatten(), labels[1:])):
    output.clip(channels=i_anom).plot(target=ax)
    any_target.plot(target=ax, color="gray", alpha=0.5)
    target.clip(channels=i_anom).plot(target=ax)
    ax.set_title(lbl)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Hardware Implementation

Now it is time to replace the software reservoir with the neuromorphic processor. As weights, we can simply use the (unscaled) integer weights from which we also generated the weights of the software reservoir.

## Neuron arrangement

We can choose explicitely to which individual neurons on the chip the neurons from the network are mapped to. It makes sense to put different partitions of the reservoir (input expansion, excitatory, inhibitory) on different cores, so that the neuron dynamics can be adjusted individually.

Furthermore, the neurons will be assigned so they form rectangles. This makes it easier to visually identify individual neurons in cortexcontrol, the software interface to the chip.

We also need to select virtual neurons, that act as a source for the external spikes from the signal-to-spike layer. The important thing is that they should have different IDs than the hardware neurons.

In [12]:
from rockpool.devices import rectangular_neuron_arrangement

# - Reservoir neuron arangement
rectangular_arrangement = [
    # Input layer
    {"first_neuron": 4, "num_neurons": 128, "width": 8},
    # Reservoir layer I
    {"first_neuron": 256, "num_neurons": 256, "width": 16},
    # Reservoir layer II
    {"first_neuron": 768, "num_neurons": 512 - 256, "width": 16},
    # Inhibitory layer
    {"first_neuron": 516, "num_neurons": 128, "width": 8},
]

neuron_ids = []
for rectangle_params in rectangular_arrangement:
    neuron_ids += list(rectangular_neuron_arrangement(**rectangle_params))
    
# - 'Virtual' neurons (input neurons)
virtual_neuron_ids = [1, 2, 3, 12]

## Controller class
Rockpool's `DynapseControl` and `DynapseControlExtd` classes allow convenient interfacing with the hardware.

We will now load neuron and synapse parameters from a file directly onto the chip. They are chosen so that the resulting dynamics are close to that of the simulation.

The hardware parameters are often refered to as "biases", because they correspond to biases in circuits on the chip. They are not to be confused with the bias of a neuron in a neural network.

After the parameters have been set, all neurons should be quiet. Sometimes there are "hot" neurons, which fire spontaneously. We will identify those and disable them.

In [13]:
from rockpool.devices import DynapseControlExtd

# - Set up DynapseControl
controller = DynapseControlExtd()

# - Load circuit biases (which define neuron and synapse characteristics)
bias_path = "network/biases.py"
controller.load_biases(bias_path)

# Silence 'hot' neurons that fire spontaneously
silence_hot_neurons_dur = 5  # Time in seconds to listen if there are hot neurons
hot_neurons = controller.silence_hot_neurons(neuron_ids, silence_hot_neurons_dur)

dynapse_control: RPyC connection established through port 1300.
dynapse_control: Already initialized chips: [0]
dynapse_control: RPyC namespace complete.
dynapse_control: RPyC connection has been setup successfully.
DynapseControl: Initializing DynapSE
DynapseControl: Spike generator module ready.
DynapseControl: Poisson generator module ready.
DynapseControl: Time constants of cores [0 1 2 3] have been reset.
DynapseControl: Neurons initialized.
	 1023 hardware neurons and 1023 virtual neurons available.
DynapseControl: Neuron connector initialized
DynapseControl: Connectivity array initialized
DynapseControl: FPGA spike generator prepared.
DynapseControl ready.
DynapseControl: Biases have been loaded from network/biases.py.
DynapseControl: Collecting IDs of neurons that spike within the next 5 seconds
DynapseControl: Generated new buffered event filter.
DynapseControl: 18 neurons spiked: [302, 303, 400, 420, 453, 536, 585, 601, 618, 681, 696, 825, 851, 942, 953, 986, 1009, 1017]
Dyna

## Hardware reservior class
There is a rockpool layer, `RecDynapSE`, that directly sets up a reservoir on the hardware. It can be used just like the regular layers.

The timestep `dt_hardware` determines the temporal resolution of data sent to the chip but does not affect computation time. Therefore we can give it a lower value.

In [14]:
from rockpool.layers import RecDynapSE

# Hardware timestep
dt_hardware = 0.1 * reservoir.dt

# - Set up hardware reservoir layer
reservoir_hw = RecDynapSE(
    weights_in=weights_res_in,
    weights_rec=weights_rec,
    neuron_ids=neuron_ids,
    virtual_neuron_ids=virtual_neuron_ids,
    dt=dt_hardware,
    controller=controller,
    clearcores_list=[0,1,2,3],
    name="hardware",
)

RecDynapSE `hardware`: Superclass initialized
DynapseControl: Connections to cores [0 1 2 3] have been cleared.
Layer `hardware`: Layer neurons allocated
Layer `hardware`: Virtual neurons allocated
DynapseControl: Excitatory connections of type `FAST_EXC` between virtual and hardware neurons have been set.
DynapseControl: Inhibitory connections of type `FAST_INH` between virtual and hardware neurons have been set.
Layer `hardware`: Connections to virtual neurons have been set.
DynapseControl: Excitatory connections of type `FAST_EXC` between hardware neurons have been set.
DynapseControl: Inhibitory connections of type `FAST_INH` between hardware neurons have been set.
Layer `hardware`: Connections from input neurons to reservoir have been set.
DynapseControl: Excitatory connections of type `SLOW_EXC` between hardware neurons have been set.
DynapseControl: Inhibitory connections of type `FAST_INH` between hardware neurons have been set.
DynapseControl: Connections have been written to 

## Readout and spike encoder

The readout and signal-to-spike-encoding layer can be exactly the same as for the software simulation. We will create an identical copy of the software readout so that they can have different weights.

In [15]:
# - Separate readout layer for hardware (for different readout weights)
readout_hw = FFExpSyn(
    weights=np.zeros((reservoir_hw.size, NUM_ANOM_CLASSES)),
    bias=0,
    dt=DT_ECG,
    tau_syn=0.175,
    name="readout"
)

In [16]:
sw_net.reset_all()
hw_net = Network(spike_enc, reservoir_hw, readout_hw, dt=DT_ECG)

## Training

Training and inference work the same way as with the software layer.

In [ ]:
# # - Generator that yields batches of ECG data
# batch_gen = data_loader.get_batch_generator(num_beats=100, batchsize=batchsize_training)

# for batch in batch_gen:
#     output = hw_net.evolve(batch.input)
#     readout_hw.train_rr(
#         batch.target,
#         output["hardware"],
#         is_first=batch.is_first,
#         is_last=batch.is_last,
#         regularize=regularize,
#     )
# hw_net.reset_all()

# np.save("weights/readout_weights_hw.py", readout_hw.weights)
# np.save("weights/readout_bias_hw.py", readout_hw.bias)

In [17]:
# - Load pre-trained weights
readout_hw.weights = np.load("network/readout_weights_hw.npy")[:, :4]
readout_hw.bias = np.load("network/readout_bias_hw.npy")[:4]

## Inference

In [18]:
# - Generator that yields batches of ECG data
num_beats = 100
ecg_data = data_loader.get_single_batch(num_beats=num_beats)

ecg_data.input.plot()

net_data = hw_net.evolve(ecg_data.input)
hw_net.reset_all()
output = net_data["readout"]
print("Ready")

Network: Evolving layer `spike_encoder` with external input as input
Network: Evolving layer `hardware` with spike_encoder's output as input
Layer `hardware`: Current batch input: 88.662648 s (638320 timesteps), 8760 events, from 0.0 s to 88.662648 s of 88.662648 s
DynapseControl: Generating FPGA event list from arrays.
DynapseControl: Stimulus prepared from arrays.
DynapseControl: Stimulus preloaded.
DynapseControl: Updated existing buffered event filter.
DynapseControl: Started stimulation for 88.662648 s.
	Recorded 2887365 event(s) and 1 trigger event(s)
		 Using trigger event 0
DynapseControl: Extracted event data
Layer `hardware`: Received event data
Layer `hardware`: Evolution successful.
Network: Evolving layer `readout` with hardware's output as input
Ready


In [22]:
%matplotlib widget

target = ecg_data.target
any_target = TSContinuous(target.times, np.any(target.samples, axis=1))

fig, axes = plt.subplots(2, 2, figsize=(8, 6))

plt.subplots_adjust(
    top=0.95, bottom=0.05, left=0.15, right=0.95, hspace=0.5, wspace=0.2
)

for i_anom, (ax, lbl) in enumerate(zip(axes.flatten(), labels[1:])):
    output.clip(channels=i_anom).plot(target=ax)
    any_target.plot(target=ax, color="gray", alpha=0.5)
    target.clip(channels=i_anom).plot(target=ax)
    ax.set_title(lbl)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …